# Project 2: Revamping the Basic Recommender System

In [6]:
import pandas as pd 
import numpy as np
import math
import csv
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine


## Purpose

### Data Collection and Cleaning

In [7]:
#Data file containing ratings data
f = open("data/Movies.txt")
text = csv.reader(f, delimiter = "\t")
ratings = list(text)
myMovieRatings = pd.DataFrame(ratings, columns = ["Movie_id", "Title"])
rate = [4,5,5,4,2,2,2,1,4,4,5,3,2,2,3,4,2,5,2,1]
myratings = pd.DataFrame(rate, columns =["Rating"])
dfs = [myMovieRatings,myratings]
myMovieRatings = pd.concat(dfs, axis = 1)
#Data file containing ratings data
f = open("data/u.data")
text = csv.reader(f, delimiter = "\t")
ratings = list(text)

# My Ratings of the movie Data (I left out the ones I wanted to "guess")
my_ratings = [4,5,5,4,0,2,2,1,4,4,5,3,2,2,3,0,2,5,2,1]

ratings_pd = pd.DataFrame(ratings, columns = ["user_id","movie_id", "rating","timestamp"])
ratings_pd = ratings_pd.drop(["timestamp"], 1)

movies = ratings_pd.pivot(index = "movie_id", columns = "user_id", values = "rating")

#Movie IDs
i = ['55','56','64','71','94','102','118','133','141','168','173','196','1278','928','780',
     '651','514','483','432','404']
movies = movies.loc[i,:]

#Random User IDs
l = ['13','128','201','207','222','23','234','246','267','268','269','276','493','642',
     '655','846','95','262','194','130','1']
movies = movies.loc[:,l]
movies = movies.fillna(value=0)

movies_np = np.array(movies)
movies_np = movies_np.astype(float)
movies_np = np.insert(movies_np, 0, my_ratings, axis=1)



### Processing Data: Normalization Vs. Non-Normalized Data

In [8]:
def mean_user(movies_np):
    movies_np = np.nan_to_num(movies_np)
    count_user = []
    mean_user = []

    for row in movies_np.T:
        row_sum = np.sum(row)
        count = np.count_nonzero(row)
        count_user.append(count)
        mean_user.append(row_sum/count)
    
    return mean_user


def norm_mean_cent(movies_np):
    mean_movie= []
    
    count_movie = []

    for row in movies_np:
        row_sum = np.sum(row)
        count = np.count_nonzero(row)
        count_movie.append(count)
        mean_movie.append(row_sum/count)
    
    count_user = []
    mean_user = []

    for row in movies_np.T:
        row_sum = np.sum(row)
        count = np.count_nonzero(row)
        count_user.append(count)
        mean_user.append(row_sum/count)

    movies_np[movies_np==0] = np.nan

    mean_cent = []
    i = 0
    for row in  movies_np:
        mean_cent.append(row - mean_movie[i]) 
        i += 1
    
    mean_cent = np.array(mean_cent)
    mean_cent = np.nan_to_num(mean_cent)
    
    return mean_cent
mean_cent = norm_mean_cent(movies_np)
mean_user = mean_user(movies_np)

### Processing Data: Item-Item Vs. User-User

In [9]:
def Item_Item(data,mean_user, user, movie):
    data = np.nan_to_num(data)
    num = 1 - pairwise_distances(data, metric="cosine")
    Sort = np.array(num[movie])
    ind = Sort.argsort()[-3:][::-1]
    x1 = data[ind[1],user]
    x2 = data[ind[2],user]    
    n = 0
    if x1 == 0:
        n = (x2 * Sort[ind[2]])/Sort[ind[2]]
    elif x2 == 0:
        n = (x1 * Sort[ind[1]])/Sort[ind[1]]
    elif x1 and x2 == 0:
        pass
    else:
        n = ((x1 * Sort[ind[1]]) + (x2 * Sort[ind[2]]))/(Sort[ind[1]]+Sort[ind[2]])

    Fin_Rating = mean_user[user] + n
    
    return Fin_Rating

def User_User(data, mean_user, user, movie):
    data = np.nan_to_num(data)
    num = 1 - pairwise_distances(data, metric="cosine")
    Sort = np.array(num[movie])
    ind = Sort.argsort()[-3:][::-1]
    x1 = data[ind[1],user]
    x2 = data[ind[2],user]
    n = 0
    if x1 == 0:
        n = (x2 * Sort[ind[2]])/Sort[ind[2]]
    elif x2 == 0:
        n = (x2 * Sort[ind[2]])/Sort[ind[2]]
    elif x1 and x2 == 0:
        pass
    else:
        n = ((x1 * Sort[ind[1]]) + (x2 * Sort[ind[2]]))/(Sort[ind[1]]+Sort[ind[2]])
    
    Fin_Rating = mean_user[user] + n
    return Fin_Rating


In [10]:
norm_item = []
for i in range(20):
    norm_item.append(Item_Item(mean_cent.T,mean_user,0,i))
norm_user = []
for i in range(20):
    norm_user.append(User_User(mean_cent,mean_user,0,i))
movie_item = []
for i in range(20):
    movie_item.append(Item_Item(movies_np.T,mean_user,0,i))
movie_user = []
for i in range(20):
    movie_user.append(User_User(movies_np,mean_user,0,i))

In [11]:
norm_item = pd.DataFrame(norm_item, columns =["norm_item"])
norm_user = pd.DataFrame(norm_user, columns =["norm_user"])
movie_item = pd.DataFrame(movie_item, columns =["movie_item"])
movie_user = pd.DataFrame(movie_user, columns =["movie_user"])

dfs = [myMovieRatings,norm_item,norm_user,movie_item,movie_user]
myMovieRatings = pd.concat(dfs, axis = 1)
myMovieRatings


,Movie_id,Title,Rating,norm_item,norm_user,movie_item,movie_user
0,55,"Professional, The (1994)",4,3.111111,2.483412,7.617014,8.111111
1,56,Pulp Fiction (1994),5,2.631243,3.182540,8.111111,7.622995
2,64,"Shawshank Redemption, The (1994)",5,4.182540,3.016200,3.111111,7.629036
3,71,"Lion King, The (1994)",4,3.544901,2.453312,3.111111,7.617519
4,94,Home Alone (1990),2,2.182540,1.808863,6.088004,5.628149
5,102,"Aristocats, The (1970)",2,3.529863,2.843482,3.111111,7.609761
6,118,Twister (1996),2,1.182540,2.524113,7.611579,6.604320
7,133,Gone with the Wind (1939),1,3.182540,2.441165,3.111111,8.111111
8,141,"20,000 Leagues Under the Sea (1954)",4,1.182540,3.571516,7.588899,6.613699
9,168,Monty Python and the Holy Grail (1974),4,4.182540,2.714354,7.609126,7.606083


### Referrences: 

# F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages. http://dx.doi.org/10.1145/2827872 http://grouplens.org/datasets/movielens/